In [1]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import numba
import os
root_folder = os.getcwd()

In [2]:
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate9900.off"))
### Parameters for running computation
global Kb
global Kv
global Ka
global gamma
global KbT
global delT
global Kal

gamma=2
Ka=2
Kv=10
Kb=0.01
H0=0
Volume_t= 0.62* 3.14 * 4 / 3
Area_t=4*3.14
KbT=0.01
charTimeStep=0.05
isAdaptiveStep=False
#hdt=0.5*dt
iterations = 15000
outfrequency = 100
tolerance = 1e-7
maxError = 2000

logfrequency=100
###Upsample_Here
#v_init,f=igl.upsample(v_init, f,1)
#v_init,f=igl.loop(v_init, f,1)
len(v_init)


import optimesh
# You can use all methods in optimesh:
# v_init, f = optimesh.optimize_points_cells(
#     v_init, f, "CVT (block-diagonal)", 1.0e-1, 100)
mp.jupyter()
p=mp.plot(v_init,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
"width": 300, "height": 300},return_plot=True)
max(v_init[:,0])
len(f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

5120

In [3]:
x1=v_init[:,0]
y1=v_init[:,1]
z1=v_init[:,2]
print(max(x1))
#v_init,f=igl.loop(v_init,f,1)
# mesh = meshplex.MeshTri(v_init,f)
# v_init = optimesh.get_new_points(mesh, "CVT (block-diagonal)")
# mp.jupyter()
# p=mp.plot(v_init,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
# "width": 300, "height": 300},return_plot=True)
len(v_init[:,1])

1.21757939188117


2562

In [4]:
###Energy Calculations /// Area+Bending+Volume
# @numba.jit
def Energy_area(v,f,Area_t):
    Area_new=cal_areatot(v,f)
    Energy_Area=Ka*((Area_new-Area_t)**2)/Area_t
    return Energy_Area
# @numba.jit
def Energy_volume(v,f,Volume_t):
    volume_new=cal_volumetot(v,f)
    Energy_volume=Kv*((volume_new-Volume_t)**2)/Volume_t
    return Energy_volume
# @numba.jit
# def Energy_bending(v,f,H0):
#     npv = igl.per_vertex_normals(v, f)
#     m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
#     minv = sp.sparse.diags(1 / m.diagonal())
#     area_voronoi=m.diagonal()
#     l = igl.cotmatrix(v, f) ###laplacian-operator
#     Hn = -minv.dot(l.dot(v))/2
#     H_mean = np.linalg.norm(Hn, axis=1)
#     sign_H = np.sign(np.sum(Hn*npv, axis=1))
#     H_mean_signed = H_mean*sign_H
#     #print(max(h_mean))
#     #print(min(h_mean))
#     Eb = 2*Kb*(((H_mean_signed-H0)**2)+K)*area_voronoi
#     total_EB = np.sum(Eb)
#     return total_EB
  

In [5]:
def adjacent_face(v,f):
    df=pd.DataFrame(f,columns=list('ABC'))
    row_numbers=[]
    for i in range(len(v)):
        row_numbers.append((df.index[(df['A'] == i)|(df['B'] == i) | (df['C'] == i)].tolist()))
    return row_numbers

def cal_areatot(v,f):
    dbl_area = igl.doublearea(v, f)    
    Areatot = np.sum(dbl_area)/2
    return Areatot
@numba.jit
def cal_volumetot(v,f):
    Volumetot = 0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volumetot+=sum
    return Volumetot

def areaGrad(v,f):
    #n=igl.per_vertex_normals(v,f) ## not using per_vertex_normals for areaGrad direction
    l = igl.cotmatrix(v, f) ###laplacian-operator
    ag = -l.dot(v)
    return ag
def volGrad(v,f):
    npv = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,npv)
    dbl_area = igl.doublearea(v, f)
    adjacent_vertices=igl.adjacency_list(f)
    adjacent_faces=adjacent_face(v,f)
    volumegrad=[]
    for i in range(len(v)):
        vol_ij=0
        for j in range(len(adjacent_faces[i])):
            k=adjacent_faces[i][j]
            Area=dbl_area[k]/2
            FaceNorm=face_normal[k]
            vol_ij += (1/3)*Area*FaceNorm
        volumegrad.append(vol_ij)
    return np.array(volumegrad)
def ver_new(v,f):
    adjacent_faces=adjacent_face(v,f)
    Area=igl.doublearea(v,f)
    n = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,n)
    v_b=igl.barycenter(v,f)
    v_new=[]
    for i in range(len(v)):
        face_area=Area[adjacent_faces[i]]
        face_area_sum=np.sum(face_area)
        v_centroid=v_b[adjacent_faces[i]]
        sum_of_area_centroid=np.dot(face_area,v_centroid)
        v_avg=sum_of_area_centroid/face_area_sum
        fnorm=face_normal[adjacent_faces[i]]
        fsum=np.sum(fnorm,axis=0)
        lamda=(np.dot(v_avg,fsum)-np.dot(v[i],fsum))/np.dot(fsum,fsum)
        v_new.append(v_avg-lamda*fsum)    
    return np.array(v_new)

In [6]:
###Force  and Energy Calculations from bending
def Force_Bending(v,f):
    npv = igl.per_vertex_normals(v, f)
    K = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    Hn = -minv.dot(l.dot(v))/2
    H_mean = np.linalg.norm(Hn, axis=1)
    sign_H = np.sign(np.sum(Hn*npv, axis=1))
    H_mean_signed = H_mean*sign_H
    Lap_H = minv.dot(l.dot(H_mean_signed-H0))
    kn = minv.dot(K)
    first_term = 2*(H_mean_signed-H0)*(H_mean_signed**2 + H0*H_mean_signed -kn)
    totalforce = first_term + Lap_H
    Force_bending = 2*Kb*npv*totalforce[:,None]*area_voronoi[:,None]

    Eb = 2*Kb*(((H_mean_signed-H0)**2))*area_voronoi
    total_EB = np.sum(Eb)
        
    return Force_bending,total_EB

In [7]:
###Force from Area Constraints
@numba.jit
def Force_Area(Area_t,grad_Area,Area_new):
    Force_Area = -2*(Ka)*((Area_new-Area_t)/Area_t)*grad_Area
    ## Area constraint is not the same as in mem3dg
    Energy_Area = Ka*(Area_new-Area_t)**2/Area_t
    return Force_Area,Energy_Area

In [8]:
###Force from Volume Area Constraints

@numba.jit
def Force_Volume(Volume_t,grad_Volume,Volume_new):
    Force_Volume=-2*(Kv)*((Volume_new-Volume_t)/Volume_t)*grad_Volume 
    ## Volume constraint is not the same as in mem3dg
    Energy_Volume = Kv*(Volume_new-Volume_t)**2/Volume_t
    return Force_Volume,Energy_Volume

In [9]:
def Total_Force(FB,FA,FV):
    Total_force=(FB+FA+FV)
    return Total_force

In [10]:
def updateTimeStep(v,f,l,TF,initialMaxForce,dt_size2_ratio,charTimeStep):
    currentMinSize = np.amin(l)
    currentMaxForce = np.amax(np.linalg.norm(TF, axis=1))
    dt = (dt_size2_ratio * currentMinSize **2)*\
        (initialMaxForce / currentMaxForce)
    
    if (charTimeStep / dt > 1e3):
        print("Time step too small! May consider restarting\n",
              "simulation in small time scale")
        print("Current size / initial size =",
              currentMinSize / sqrt(charTimeStep/dt_size2_ratio))
        print("Current forece / inital force =",
             currentMaxForce / initialMaxForce)
        exit()
    return dt

In [ ]:
### main loop for simulation
l = igl.edge_lengths(v_init,f)
l0 = igl.avg_edge_length(v_init,f)
dt_size2_ratio = charTimeStep / np.amin(l)**2
number_of_vertices=len(v_init)
###Forward Euler Main 
#Setup

v=v_init
vel=np.zeros((len(v_init),3))
time=0
#FD=fun_FD2(v_init,f,vel)

#Volume_old=cal_volume(v,f)
#Area_t=cal_Area(v,f)
grad_Area=areaGrad(v_init,f)
grad_Volume=volGrad(v_init,f)
Area_new=cal_areatot(v_init,f)
print('initial area = ',Area_new)
Volume_new=cal_volumetot(v_init,f)
print('initial volume = ', Volume_new)

FB,EB=Force_Bending(v_init,f)
FA,EA=Force_Area(Area_t,grad_Area,Area_new)
FV,EV=Force_Volume(Volume_t,grad_Volume,Volume_new)
#Fad,Ead=Force_Adhesive(v_init,f,x,y,z,rho,U,grad_Area)
#FR=Force_Random(v,f)
TF=Total_Force(FB,FA,FV)#,Fad)
initialMaxForce = np.amax(np.linalg.norm(TF, axis=1))
Etot_old = EB+EA+EV#+Ead

### Calculation of Energy
timeout=[]
totalEnergy=[]
EnergyArea=[]
EnergyVolume=[]
EnergyBending=[]
EnergyAdhesive=[]

for i in range(iterations):
    #Integration
    
    vel = TF/gamma
    
    ## adjust time step if adopt adaptive time step based on mesh size
    if (isAdaptiveStep):
        charTimeStep = updateTimeStep(v,f,l,TF,initialMaxForce,
                                                dt_size2_ratio,charTimeStep);
        
    dt = charTimeStep
    
    v += vel*dt
    time += dt
    
    ###Mesh Regularization
    l = igl.edge_lengths(v,f)
    v=ver_new(v,f)
    if (not igl.is_intrinsic_delaunay(l,f).all()):
        #v,f,_=pymesh.split_long_edges_raw(v,f,l0*4/3)
        print('time =',time,': not all edges are delaunay, call intrinsic delaunay triangulation')
        l, f= igl.intrinsic_delaunay_triangulation(l,f)
   
    #Force calculation
    Area_current=cal_areatot(v,f)
    grad_Area_new=areaGrad(v,f)
    Volume_current=cal_volumetot(v,f)
    grad_Volume_new=volGrad(v,f)
    ###Update forces here
 
    FB,EB=Force_Bending(v,f) ##bending_force and Bending Energy
    FA,EA=Force_Area(Area_t,grad_Area_new,Area_current)
    FV,EV=Force_Volume(Volume_t,grad_Volume_new,Volume_current)
#    Fad,Ead=Force_Adhesive(v,f,x,y,z,rho,U,grad_Area_new)

    TF=Total_Force(FB,FA,FV)#,Fad)
    Etot_new = EB+EA+EV#+Ead
    
    mechErrorNorm = np.sum(np.linalg.norm(TF, axis=1))
    
    EnergyChange = abs(Etot_new-Etot_old)
    Etot_old = Etot_new
    
    if EnergyChange<tolerance:
        print('Change of Energy is very small \n Reached Equilibrioum Shape')
        print('time =',time)
        print('dt =',dt)
        print('dVolume/Volume_t =',Volume_current-Volume_t,'/',Volume_t)
        print('dArea/Area_t =',Area_current-Area_t,'/',Area_t)
        print('mechErrorNorm =',mechErrorNorm)
        timeout.append(time)
        EnergyArea.append(EA)
        EnergyVolume.append(EV)
        EnergyBending.append(EB)
        #EnergyAdhesive.append(Ead)
        PotentialEnergy = np.array(EnergyArea) + np.array(EnergyVolume) + np.array(EnergyBending)#+np.array(EnergyAdhesive)   
        print('PE = ',PotentialEnergy[-1])
        print('EnergyChange = ',EnergyChange)
        print('number of vertices = ',len(v))
        print('number of iterations = ',i)

        
        break
    
    if i%logfrequency==0:
        print('time =',time)
        print('dt =',dt)
        print('dVolume/Volume_t =',Volume_current-Volume_t,'/',Volume_t)
        print('dArea/Area_t =',Area_current-Area_t,'/',Area_t)
        print('mechErrorNorm =',mechErrorNorm)
        timeout.append(time)
        EnergyArea.append(EA)
        EnergyVolume.append(EV)
        EnergyBending.append(EB)
#        EnergyAdhesive.append(Ead)
        PotentialEnergy = np.array(EnergyArea) + np.array(EnergyVolume) + np.array(EnergyBending)
         #+np.array(EnergyAdhesive)   
        print('PE = ',PotentialEnergy[-1])
        print('EnergyChange = ',EnergyChange)
        print('number of vertices = ',len(v))
        print('number of iterations = ',i)



initial area =  12.413540591680148
initial volume =  2.6153409645513053
time = 0.05
dt = 0.05
dVolume/Volume_t = 0.01963347687343786 / 2.5957333333333334
dArea/Area_t = -0.14643918410545176 / 12.56
mechErrorNorm = 61.82177773121359
PE =  0.6178253999461663
EnergyChange =  0.001884976181518283
number of vertices =  2562
number of iterations =  0
time = 5.04999999999999
dt = 0.05
dVolume/Volume_t = 0.019632349156367468 / 2.5957333333333334
dArea/Area_t = -0.1462863609363847 / 12.56
mechErrorNorm = 61.7748872858169
PE =  0.6202897410175371
EnergyChange =  0.00023242232875242408
number of vertices =  2562
number of iterations =  100
time = 10.050000000000008
dt = 0.05
dVolume/Volume_t = 0.019633233196381106 / 2.5957333333333334
dArea/Area_t = -0.14630748554663775 / 12.56
mechErrorNorm = 61.75831233880352
PE =  0.6202516128750182
EnergyChange =  0.00023160520377085714
number of vertices =  2562
number of iterations =  200
time = 15.050000000000079
dt = 0.05
dVolume/Volume_t = 0.019633977330

In [ ]:
time=np.linspace(0,iterations,len(EnergyBending))
# plt.plot(time,totalEnergy,'o-', color='red', label='total Energy')
plt.plot(time,EnergyBending,'o-', label='Bending Energy')
plt.plot(time,EnergyVolume,'v-', label='Volume Energy')
plt.plot(time,EnergyArea,'o-', label='Area Energy')
plt.plot(time,PotentialEnergy,'x-',label='Potential Energy')
# plt.plot(time,KineticEnergy,'v-', label='Kinetic Energy',color='k')
#plt.axis([0, 800, 0, 2])
plt.legend(loc='upper right',frameon=False)
plt.title('timesteps vs Energy')
plt.show()

x=[0.67,0.70,0.85,0.95]
y=[0.538019459/(8*3.14*0.01),0.49591257/(8*3.14*0.01) ,0.3386189912583/(8*3.14*0.01),0.27790931351/(8*3.14*0.01)]


X=[0.9595367804007768,0.9258040599051776,0.871844218733256,0.8371499569913026,0.8014918370361359,0.7272793885524458,0.688720426090205,0.6289635338378528,0.5875083628022554]
Y=[1.0736093130482884,1.1556386284714684,1.2738734472701787,1.3559056589019316, 1.4403504122752906,1.6212968353128918, 1.7226177960431999, 1.8697857384897432,1.983163615510935]
# plt.plot(X, Y)
# plt.show()
#plt.hold(True)
plt.axis([0.4, 1, 1, 2.5])
plt.plot(x,y,'g^',X,Y,'r--')
plt.xlabel('reduced volume (v)')
plt.legend(['Euler_simulation', 'Theory(Seifert et al.)'])

plt.ylabel('Bending Energy (BE)/(8 pi Kb)')

plt.show()

In [ ]:
Volume_Final= cal_volumetot(v,f)
print(Volume_Final)
Volume_Final/Volume_new

In [ ]:
EnergyBending

In [ ]:
mp.jupyter()
p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
"width": 300, "height": 300},return_plot=True)

In [ ]:
#igl.write_triangle_mesh(os.path.join(root_folder,'oblate9900.off'), v, f)